In [143]:
# imports
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [144]:
# params
calib_methods = ["RF"] #cal.calib_methods.copy() 
metrics = ["acc", "tce"]#cal.metrics.copy()

plot = True
 
params = {
    "runs": 1,
    "n_tree": [2, 5, 10, 20], 
    "data_size": 10000,
    "n_features": 2,
    "oob": False,
    "test_split": 0.3,
    "calib_split": 0.1
}

In [145]:
data_list = []

X, y, tp = dp.make_classification_gaussian_with_true_prob(params["data_size"], params["n_features"], 
                                                          class1_mean_min=0, 
                                                          class1_mean_max=1,

                                                          class2_mean_min=1, 
                                                          class2_mean_max=3, 

                                                          seed=0)

# plot data
# plt.scatter(X[:,0], X[:,1], c=y)
# plt.show()

In [146]:
calib_results_dict = {}
data_dict = {} # results for each data set will be saved in here.
for exp_trees in params["n_tree"]:

    # Data
    exp_data_name = str(exp_trees)
    data_list.append(exp_data_name)

    for seed in range(params["runs"]): # running the same dataset multiple times
        # split the data
        data = cal.split_train_calib_test(exp_data_name, X, y, params["test_split"], params["calib_split"], seed, tp)

        # train model
        irrf = IR_RF(n_estimators=exp_trees, oob_score=params["oob"], random_state=seed)
        irrf.fit(data["x_train"], data["y_train"])

        # calibration
        res = cal.calibration(irrf, data, calib_methods, metrics) # res is a dict with all the metrics results as well as RF probs and every calibration method decision for every test data point
        
        lr = LogisticRegression(random_state=0).fit(data["x_train"], data["y_train"])
        lr_p_test = lr.predict_proba(data["x_test"])
        res[data["name"] + "_LR_prob"] = lr_p_test
        res[data["name"] + "_LR_decision"] = np.argmax(lr_p_test, axis=1)
        res[data["name"] + "_LR_acc"] = accuracy_score(data["y_test"], res[data["name"] + "_LR_decision"])
        res[data["name"] + "_LR_tce"] = mean_squared_error(data["tp_test"], res[data["name"] + "_LR_prob"][:,1])

        svm = SVC(probability=True, random_state=0).fit(data["x_train"], data["y_train"])
        svm_p_test = svm.predict_proba(data["x_test"])
        res[data["name"] + "_SVM_prob"] = svm_p_test
        res[data["name"] + "_SVM_decision"] = np.argmax(svm_p_test, axis=1)
        res[data["name"] + "_SVM_acc"] = accuracy_score(data["y_test"], res[data["name"] + "_SVM_decision"])
        res[data["name"] + "_SVM_tce"] = mean_squared_error(data["tp_test"], res[data["name"] + "_SVM_prob"][:,1])


        data_dict = cal.update_runs(data_dict, res) # calib results for every run for the same dataset is aggregated in data_dict (ex. acc of every run as an array)
        
        if plot:
            # plot RF
            cal.plot_probs(exp_data_name, res, data, calib_methods, seed, True) 

            # plot LR
            plt.plot([0, 1], [0, 1], linestyle='--')
            colors = ['black', 'red']
            plt.scatter(data["tp_test"], lr_p_test[:,1], marker='.', c=[colors[c] for c in data["y_test"].astype(int)])
            plt.scatter(data["tp_test"], res[f"{exp_data_name}_RF_prob"][:,1], marker='.', c=[colors[c] for c in data["y_test"].astype(int)], alpha=0.1)
            plt.xlabel("True probability")
            plt.ylabel("Predicted probability")
            red_patch = plt.plot([],[], marker='o', markersize=10, color='red', linestyle='')[0]
            black_patch = plt.plot([],[], marker='o', markersize=10, color='black', linestyle='')[0]
            calib_patch = plt.plot([],[], marker='_', markersize=15, color='blue', linestyle='')[0]
            plt.legend((red_patch, black_patch, calib_patch), ('Class 0', 'Class 1', "LR"))
            path = f"../../results/Synthetic/plots/{seed}/LR"
            if not os.path.exists(path):
                os.makedirs(path)
            plt.savefig(f"{path}/LR_{exp_data_name}.png")
            plt.close()

            # plot LR
            plt.plot([0, 1], [0, 1], linestyle='--')
            colors = ['black', 'red']
            plt.scatter(data["tp_test"], svm_p_test[:,1], marker='.', c=[colors[c] for c in data["y_test"].astype(int)])
            plt.scatter(data["tp_test"], res[f"{exp_data_name}_RF_prob"][:,1], marker='.', c=[colors[c] for c in data["y_test"].astype(int)], alpha=0.1)
            plt.xlabel("True probability")
            plt.ylabel("Predicted probability")
            red_patch = plt.plot([],[], marker='o', markersize=10, color='red', linestyle='')[0]
            black_patch = plt.plot([],[], marker='o', markersize=10, color='black', linestyle='')[0]
            calib_patch = plt.plot([],[], marker='_', markersize=15, color='blue', linestyle='')[0]
            plt.legend((red_patch, black_patch, calib_patch), ('Class 0', 'Class 1', "SVM"))
            path = f"../../results/Synthetic/plots/{seed}/SVM"
            if not os.path.exists(path):
                os.makedirs(path)
            plt.savefig(f"{path}/SVM_{exp_data_name}.png")
            plt.close()

        calib_results_dict.update(data_dict) # merge results of all datasets together


KeyError: '2_RF_acc'

In [ ]:
calib_methods.append("LR")   
calib_methods.append("SVM")   
tables = cal.mean_and_ranking_table(calib_results_dict, metrics, calib_methods, data_list, mean_and_rank=False)

In [ ]:
tables["acc"]

,RF,LR,SVM
Data,,,
S,0.764,0.795,0.793


In [ ]:
tables["tce"]

,RF,LR,SVM
Data,,,
S,0.019014,0.000391,0.007113
